In [70]:
import json
import jsonlines
from collections import Counter
from functools import reduce
import numpy as np
import re
from os import listdir
from os import close
import codecs 
import mediawiki
from joblib import Parallel, delayed
import sys
from time import sleep
import os

wikipedia = mediawiki.MediaWiki(lang=u'ru')

In [71]:
import urllib.request
from urllib.parse import quote

def request_categories(ids):
    url = "https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids="
    url = url + '|'.join(ids)
    print(url)
    try:
        p = urllib.request.urlopen(url).read()
    except KeyboardInterrupt:
        sys.exit(0)
            
    except:
        print("Url not opened")
        sleep(30)
        return {}
    
    pages = json.loads(p)['query']['pages']
    result = {}
    for id in ids:
        try:
            article = pages[id]['revisions'][-1]['*']
            categories = re.findall(r'\[\[Категория:(.*)\]\]', article)
            if len(categories) > 0:
                for i in range(len(categories)):
                    categories[i] = re.sub(r'^\s', '', categories[i])
                    categories[i] = re.sub(r'(\|\s)$', '', categories[i])
                    
                categories = sorted(list(categories))
                result[id] = categories
            #else:
                #print(id + ' - no categories')
                
        except KeyboardInterrupt:
            sys.exit(0)
            
        except:
            print(id + " is gone")
    
    return result

In [72]:
request_categories(["71", "76"])

https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=71|76


{'71': ['Города Древней Руси',
  'Города из Списка русских городов дальних и ближних',
  'Города федерального значения России',
  'Города, в которых проходили летние Олимпийские игры 1980',
  'Города, основанные в XII веке',
  'Города-миллионеры России',
  'Золотое кольцо России',
  'Москва',
  'Населённые пункты, основанные в 1147 году',
  'Областные центры России',
  'Столицы летних Универсиад']}

In [73]:
path = 'wiki_extracted1/'
alf = listdir(path)

In [2]:
%%capture

'''
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

        
def walker(a):
    data = {}
    _path = path + a
    __path = [_path + '/' + p for p in listdir(_path)]
    print(a, end=' ')
    output = jsonlines.open('c/' + a, 'w')
    for p in __path:
        print(p) #[18:])
        with jsonlines.open(p, 'r') as f:
            file_data = {}
            ids = []
            for article in f:
                id = article['id']
                ids.append(id)
                file_data[id] = {'title': article['title'], 'text': article['text']}
            
            for chunk in chunks(ids, 20):
                result = request_categories(chunk)
                for k in result.keys():
                    file_data[k].update({'categories': result[k]})
                sleep(2)
                
            #print(file_data.keys())
            file_data = {key: value for key, value in file_data.items() if 'categories' in value.keys()}
            
            #print(file_data.keys())
            '''
            file = re.sub(r'\n+', '. ', file)
            file = re.split(r'</doc>', file)
            for f in file[:-1]:
                id = re.findall(r'id="(.*)" url', f)[0]
                try:
                    categories.extend(wikipedia.page(pageid=int(id)).categories)
                    page_ids.append(int(id))
                    articles.append(re.findall(r'">\. (.*)', f)[0])
                except KeyboardInterrupt:
                    close()
                except:
                    print("Page with id " + str(id) + " not found")
            '''
            
            #Тут немного намудрил, но исправлять предыдущее лень
            for d in file_data.items():
                o = d[1]
                o['id'] = d[0]
                output.write(o)
            #data.update(file_data)
    
    #return data

#for a in alf:
    #walker(a)
print(alf)
Parallel(n_jobs=2)(delayed(walker)(a) for a in alf)
'''

IndentationError: unexpected indent (<ipython-input-2-79fbe78342d9>, line 35)

In [ ]:
### with open('output.json') as f:
    #info = json.load(f)
    #print(len(info))
    #categories = [i["categories"] for i in info]
    #categories = reduce(lambda x,y:x+y, categories)
categories = [c[10:] for c in categories]
print(sorted(set([c for c in categories if re.findall(r'.* /d/d/d/d .*', c)])))
counts = Counter(categories)
print(np.mean(list(counts.values())))
print(counts)

In [67]:
a = ' asdkj'
b = 'asd| '
re.sub(r'^\s', '', b)
re.sub(r'(\|\s)$', '', b)

'asd'

In [ ]:
a = {'1': 1, '2': 2}
a.get('3') = a.get('3') + 1
a